In [ ]:
import os
import yaml
import pandas as pd
from collections import defaultdict

# Paths
root_dir = r"C:\Users\isbmy\.vscode\STOCK_PROJECT\data"
output_dir = "path/final_csvs"
os.makedirs(output_dir, exist_ok=True)

# Dictionary to collect data per ticker
symbol_data = defaultdict(list)

# Walk through folders and process YAML
for foldername, _, filenames in os.walk(root_dir):
    for filename in filenames:
        if filename.endswith(".yaml"):
            filepath = os.path.join(foldername, filename)
            with open(filepath, "r") as file:
                try:
                    records = yaml.safe_load(file)
                    for record in records:
                        symbol = record.get("Ticker")
                        if symbol:
                            symbol_data[symbol].append(record)
                except Exception as e:
                    print(f"Error reading {filepath}: {e}")

# Load the sector data
sector_df = pd.read_csv(r"C:\Users\isbmy\.vscode\STOCK_PROJECT\data\Sector_data - Sheet1.csv")

# Ensure column names are consistent for merging
sector_df.rename(columns={'Symbol': 'Ticker'}, inplace=True)
sector_df['Ticker'] = sector_df['Ticker'].str.split(':').str[-1]

# Clean Ticker column (remove leading/trailing spaces)
sector_df['Ticker'] = sector_df['Ticker'].str.strip()

# Replace specific ticker names to match the records
sector_df.replace('ADANIGREEN', 'ADANIENT', inplace=True)
sector_df.replace('AIRTEL', 'BHARTIARTL', inplace=True)
sector_df.replace('TATACONSUMER', 'TATACONSUM', inplace=True)

# Add BRITANNIA data to sector_df
sector_df = pd.concat([sector_df, pd.DataFrame([{'COMPANY': 'BRITANNIA INDUSTRIES', 'Ticker': 'BRITANNIA', 'sector': 'CONSUMER GOODS'}])], ignore_index=True)
# Add company and sector columns to the existing CSVs
for symbol, records in symbol_data.items():
    df = pd.DataFrame(records)

    # Merge with sector_df to add company and sector information
    df = df.merge(sector_df[['Ticker', 'COMPANY', 'sector']], on='Ticker', how='left')

    # Split datetime into date and time
    if 'date' in df.columns:
        df['date'] = pd.to_datetime(df['date'], errors='coerce')
        df['time'] = df['date'].dt.time.astype(str)
        df['date'] = df['date'].dt.date.astype(str)

    # Save the updated CSV
    df.to_csv(os.path.join(output_dir, f"{symbol}.csv"), index=False)
    if df['COMPANY'].isnull().all():
        print(f"{symbol}.csv: The COMPANY column is empty.")
    elif df['COMPANY'].isnull().any():
        print(f"{symbol}.csv: The COMPANY column has some empty values.")
    else:
        pass

print(f"Updated CSV files with company and sector info saved in: {output_dir}")


Updated CSV files with company and sector info saved in: path/final_csvs


In [ ]:
# Connect to TiDB Cloud SQL
import mysql.connector

# Connect to TiDB Cloud SQL
connection = mysql.connector.connect(
    user='3gjpMYnFyBFAYAb.root',
    password='J9foCvujVRIFORzR',
    host='gateway01.ap-southeast-1.prod.aws.tidbcloud.com',
    port='4000'
)

if connection.is_connected():
    print("Connected to TiDB Cloud SQL")
else:
    print("Connection failed")


Connected to TiDB Cloud SQL


In [ ]:
# Create a database and table for the ticker data
import os
import pandas as pd

# Define the database and table name
database_name = "stock_data"
table_name = "ticker"

# Create a cursor object
cursor = connection.cursor()

# Create the database if it doesn't exist
cursor.execute(f"CREATE DATABASE IF NOT EXISTS {database_name}")
cursor.execute(f"USE {database_name}")

# Iterate through all CSV files in the output directory
for csv_file in os.listdir(output_dir):
    if csv_file.endswith(".csv"):
        file_path = os.path.join(output_dir, csv_file)
        df = pd.read_csv(file_path)

        # Create a table for the ticker if it doesn't exist
        create_table_query = f"""
        CREATE TABLE IF NOT EXISTS {table_name} (
            {', '.join([f'{col} TEXT' for col in df.columns])}
        )
        """
        cursor.execute(create_table_query)

        # Insert data into the table
        for _, row in df.iterrows():
            insert_query = f"INSERT INTO {table_name} VALUES ({', '.join(['%s'] * len(row))})"
            cursor.execute(insert_query, tuple(row))

# Commit the transaction and close the cursor
connection.commit()
cursor.close()
print(f"Data from CSV files saved to TiDB Cloud SQL in database '{database_name}'.")

Data from CSV files saved to TiDB Cloud SQL in database 'stock_data'.


In [ ]:
#combine all CSVs into one
import os
import pandas as pd
from glob import glob

# Folder where your individual stock CSVs are stored
folder_path = r"C:\Users\isbmy\.vscode\STOCK_PROJECT\final_csvs"

# Get all CSV files in the folder
csv_files = glob(os.path.join(folder_path, "*.csv"))

# Read and combine them
combined_df = pd.concat(
    [pd.read_csv(file).assign(Ticker=os.path.basename(file).replace(".csv", "")) for file in csv_files],
    ignore_index=True
)

# Save the combined DataFrame to a single CSV
output_path = os.path.join(folder_path, "combined_for_powerbi.csv")
combined_df.to_csv(output_path, index=False)

print(f"Combined CSV saved at: {output_path}")


Combined CSV saved at: C:\Users\isbmy\.vscode\STOCK_PROJECT\final_csvs\combined_for_powerbi.csv


In [ ]:
#Data Export for Power BI
import os
import pandas as pd
import numpy as np
from glob import glob

# Define input/output
data_folder = r"C:\Users\isbmy\.vscode\STOCK_PROJECT\final_csvs"  # Replace with your actual path
output_file = "exported_for_powerbi.csv"

# Load and combine all CSVs
df_list = []
for file in glob(os.path.join(data_folder, "*.csv")):
    try:
        temp_df = pd.read_csv(file)
        temp_df["Ticker"] = os.path.basename(file).replace(".csv", "")
        df_list.append(temp_df)
    except Exception as e:
        print(f"Error reading {file}: {e}")

# Concatenate all dataframes
df = pd.concat(df_list, ignore_index=True)

# Parse date and sort
df["date"] = pd.to_datetime(df["date"], errors="coerce")
df = df.sort_values(by=["Ticker", "date"])

# Add daily return
df["prev_close"] = df.groupby("Ticker")["close"].shift(1)
df["daily_return"] = (df["close"] - df["prev_close"]) / df["prev_close"]

# Add cumulative return
cumulative_returns = []
for ticker, group in df.groupby("Ticker"):
    group = group.copy()
    group["cumulative_return"] = (1 + group["daily_return"].fillna(0)).cumprod()
    cumulative_returns.append(group)
df = pd.concat(cumulative_returns)

# Calculate yearly return per ticker
yearly_return_df = df.groupby("Ticker").agg(
    first_price=("close", "first"),
    last_price=("close", "last")
).reset_index()
yearly_return_df["yearly_return"] = (yearly_return_df["last_price"] - yearly_return_df["first_price"]) / yearly_return_df["first_price"]

# Merge yearly return back to main dataframe
df = df.merge(yearly_return_df[["Ticker", "yearly_return"]], on="Ticker", how="left")

# Add month column
df["month"] = df["date"].dt.to_period("M").astype(str)

# Drop unnecessary columns
df.drop(columns=["prev_close"], inplace=True)

# Save to CSV
df.to_csv(output_file, index=False)
print(f"Data exported to {output_file}")


Data exported to exported_for_powerbi.csv
